# Solving Semidefinite Programs (SDPs) in Python

In this notebook, we will be looking at how one may specify and numerically solve SDPs in Python. 

You may already be aware of the [CVX software package](http://cvxr.com/cvx/) for MATLAB. While this software is powerful and useful for solving a wide variety of SDPs, it requires a MATLAB license which can be cost prohibitive. 

The [cvxpy module](https://www.cvxpy.org/index.html) in Python is an open-source option for solving convex optimization problems. The way in which one specifies an SDP in Python can be a bit awkward if you have not seen it before. In this notebook, we will cover how to specify and solve SDPs using this module. 

## Semidefinite programming

- Generalization of linear programming.
- Powerful tool with many applications in quantum information.
- SDPs are efficiently solvable in (polynomial time).
- Software packages for solving SDPs exist (`cvxpy`).

## The "standard form" for SDPs

In the computational physics course, the following form of how one may specify an SDP should look familiar to you:

$$
\begin{equation*}
    \begin{aligned}
        \underline{\text{Primal problem:}} \quad & \\
        \text{maximize:} \quad & \langle A, X \rangle \\
        \text{subject to:} \quad & \Phi(X) = B, \\
        \quad & X \in \text{Pos}(\mathcal{X}).
    \end{aligned}
\end{equation*}
$$
$$
\begin{equation*}
    \begin{aligned}
        \underline{\text{Dual problem:}} \quad & \\
        \text{minimize:} \quad & \langle B, Y \rangle \\
        \text{subject to:} \quad & \Phi^*(Y) \geq A, \\
                           \quad & Y \in \text{Herm}(\mathcal{Y}).
     \end{aligned}
\end{equation*}
$$

# Solving SDPs in Python

In order to solve SDPs in Python, we will assume you have Python 3.6+ installed on your machine along with [pip](https://www.python.org/downloads/)--Python's package manager. It's a fairly safe bet that you already have some version of Python installed, but ensure you have the proper version. You can download Python [here](https://www.python.org/downloads/).

We will also require the use of two Python modules--`cvxpy` and `numpy`. The `cvxpy` module is what we will use to specify and numerically solve our SDPs. The `numpy` module will come in handy when we want to define objects like matrices and vectros. We will shortly cover how to install the `cvxpy` and `numpy` modules.

## The `cvxpy` module

As specified on the [cvxpy website](https://www.cvxpy.org/index.html):

> The `cvxpy` module is a Python-embedded modeling language for convex optimization problems. It allows you to express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

As `cvxpy` is a general framework for convex optimization (not just semidefinite programming) the documentation is quite sparse on how to use `cvxpy` to specify and solve an SDP. Here is [the one and only example](https://www.cvxpy.org/examples/basic/sdp.html) for how to specify an SDP using `cvxpy`.

Furthermore, the example that does exist in the documentation is cast in the "non-standard" form--making the translation between paper and code a bit more difficult. 

## Installing the `cvxpy` and `numpy` modules

Assuming you have Python 3.6+  and `pip` installed, you can run the following commands to install the `cvxpy` and `numpy` modules on your machine. 

In [ ]:
pip install cvxpy

In [ ]:
pip install numpy

As a sanity check, run the following two-line Python script to ensure to errors arise. If you see no output, both modules are successfully installed and ready to use.

In [4]:
import cvxpy
import numpy

# Solving simple SDPs

Let's take a look at a simple SDP to see how we can encode it using Python and `cvxpy`.

$$
\begin{equation*}
    \begin{aligned}
        \underline{\text{Primal problem:}} \quad & \\
            \text{maximize:} \quad & \langle \mathbb{I}_4, X \rangle \\
            \text{subject to:} \quad & X = \mathbb{I}_4, \\
                               \quad & X \in \text{Pos}(\mathcal{X}).
     \end{aligned}
\end{equation*}
$$

We are going to take a look at this particular (trivial) SDP and see how we can translate the specification of it into a form that Python and `cvxpy` understands and can use.

Let us begin by simply importing the necessary `numpy` and `cvxpy` packages into our script.

In [ ]:
import cvxpy
import numpy as np

Next, we want to define a variabe, $X$, that we will be optimizing over in our example SDP. The `cvxpy` module allows us to specify objects of type `Variable` which may be scalars, vectors, matrices, etc. In this case, we want to define a variable that is a $4$-by-$4$ matrix to match the dimension of $\mathbb{I}_4$.

In [7]:
x_var = cvxpy.Variable((4, 4))

So the variable `x_var` is an object of type `Variable` that we will be optimizing over in our SDP. Next, let's define the *objective function* of our SDP--that is, the bit that looks like $\langle \mathbb{I}_4, X \rangle$. 

In our objective function, we are taking the maximum. There is a built-in method, `cvxpy.Maximum()` that we will pass what we wish to maximize into. In this case we can encoe this function as follows.

In [8]:
objective = cvxpy.Maximize(cvxpy.trace(x_var.H @ np.identity(4)))

Note that we are writing the inner product in terms of the trace function, that is, you may recall that:

$$
\langle \mathbb{I}_4, X \rangle \equiv \text{Tr}(X^* \mathbb{I}_4))
$$

where $\text{Tr}(\cdot)$ is the trace function and where we denote $X^*$ as the complex conjugate of the matrix $X$.

Another key thin to point out is how we are using the `@` operator instead of the `*` operator to denote multiplication. In more recent version of Python, it is preferred to denote scalar multiplication with the `*` operator and matrix multiplication with the `@` operator.

Next, let's define the constraints of our SDP. The `cvxpy` module expects that we provide these inside of a Python list.

In [9]:
constraints = []

We have two constraints that we need to ensure are enforced:

1. $X = \mathbb{I}_4$,
2. $X \in \text{Pos}(\mathcal{X})$.

For the first constraint, we want to enforce that the variable `x_var` is equal to the 4-by-4 identity matrix. We can add that to our list of constraints like so.

In [10]:
constraints.append(x_var == np.identity(4))

Note that the `np.identity(4)` code represents the 4-dimensional identity matrix. That is:

In [11]:
np.identity(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

Note also that we are adding to our `constraints` list that we want to enforce that the `x_var` variable is equal to the identity matrix. 

Next, we want to enforce that `x_var` is positive semidefinite. We can add this constraint to our `constraints` list as:

In [12]:
constraints.append(x_var >> 0)

Now that we have all our constraints defined in our `constraints` list, we can pass both our `objective` and `constraints` objective to an instance of the `Problem` class in `cvxpy`. 

In [13]:
problem = cvxpy.Problem(objective, constraints)

The `problem` object we return here is a `cvxpy` way of specifying our SDP. We can call the built-in method `solve` on any instance of a `Problem` object to get the corresponding optimal value of our SDP.

In [16]:
val = problem.solve()
val

4.0

We see that the optimal value provided to us from our SDP is `4`, which is exactly what we expect. Furthermore, we can also see what the solver found to be the optimal value for the variable `X`. We can extract this by calling the `.value` method on our `x_var` variable.

In [18]:
x_var.value

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

Again, no surprises here that the value of variable $X$ is simply the 4-by-4 identity matrix.

For completeness, we show the script in its entirety below:

In [ ]:
import cvxpy
import numpy as np

# Declare a 4x4 variable for our SDP.
x_var = cvxpy.Variable((4, 4))

# We can specify the constraints for our SDP in a Python list.
constraints = []

# We can write the inner product:
#    <A, X> <-> Tr(A^*, X)
objective = cvxpy.Maximize(cvxpy.trace(x_var.H @ np.identity(4)))
constraints.append(x_var == np.identity(4))
constraints.append(x_var >> 0)

# Feed in the objective and constraints to the solver.
problem = cvxpy.Problem(objective, constraints)
val = problem.solve()

# Print out the optimal value along with the optimal solution.

# The optimal value of this SDP is equal to "4" (the dimension) of the identity matrix.
print(val)

# Trivially, the variable that optimally satisfies this SDP is the 4x4 identity matrix.
print(x_var.value)

We can alter the SDP slightly so that no feasible solution exists--that is that the optimal value is $-\infty$. That is, we say that this SDP is **infeasible**.

$$
            \begin{equation*}
                \begin{aligned}
                    \underline{\text{Primal problem:}} \quad & \\
                    \text{maximize:} \quad & \langle \mathbb{I}_4, X \rangle \\
                    \text{subject to:} \quad & X = -\mathbb{I}_4, \\
                                       \quad & X \in \text{Pos}(\mathcal{X}).
                \end{aligned}
            \end{equation*}
$$

In [ ]:
import cvxpy
import numpy as np

x_var = cvxpy.Variable((4, 4))

constraints = []
objective = cvxpy.Maximize(cvxpy.trace(x_var.H @ np.identity(4)))
constraints.append(x_var == -np.identity(4))
constraints.append(x_var >> 0)

problem = cvxpy.Problem(objective, constraints)
val = problem.solve()

print(val)

# Applications of SDPs to Quantum Information

## Computing the fidelity function

**Problem**: Calculate the fidelity of two quantum states $\rho$ and $\sigma$ defined as 

$$
F(\rho, \sigma) = ||\sqrt{\rho}\sqrt{\sigma}||_1.
$$

For the sake of not diving into extraneous details we can assume the following things to make this a bit simpler:

- Take for granite (for the sake of expediency) that one can compute $F(\rho, \sigma)$ by way of a semidefinite program.

- One could of course compute this quantity without making use of an SDP, but it is often useful and convenient to frame this in terms of an SDP.
  
- Calculating the fidelity is an interesting and important problem in the field of quantum information--it serves as a measure of how "close" two quantum states are to each other.

For further simplicity (and to obscure any quantum conotation), the fidelity function is a function on density matrices (matrices that are positive semidefinite with trace equal to $1$).

For the sake of example ...

The fidelity function can be defined in terms of an SDP [[Kiloran12]](https://uwspace.uwaterloo.ca/handle/10012/6662) [[Watrous12]](https://arxiv.org/abs/1207.5726) whose primal problem is specified as:

$$
            \begin{equation*}
                \begin{aligned}
                    \underline{\text{Primal problem:}} \quad & \\
                    \text{maximize:} \quad & \frac{1}{2} \text{Tr}(X) + \frac{1}{2} \text{Tr}(X^*) \\
                    \text{subject to:} \quad & \begin{pmatrix}
                                                   \rho & X \\
                                                   X^* & \sigma
                                               \end{pmatrix} \in \geq 0
                \end{aligned}
            \end{equation*}
$$

In [ ]:
import cvxpy
import numpy as np

x_var = cvxpy.Variable(rho.shape, complex=True)
objective = cvxpy.Maximize(cvxpy.real(cvxpy.trace(z_var + z_var.H)))
# constraints = [cvxpy.bmat([[rho, z_var], [z_var.H, sigma]]) >> 0]
# problem = cvxpy.Problem(objective, constraints)

# return 1 / 2 * problem.solve()

Note that this value we obtain from the SDP is equivalent to the one we obtain from a direct calculation:

In [3]:
import numpy as np

rho = np.array([[1 / 2, 0, 0, 1 / 2], [0, 0, 0, 0], [0, 0, 0, 0], [1 / 2, 0, 0, 1 / 2]])
sigma = np.array([[1 / 2, 0, 0, 1 / 2], [0, 0, 0, 0], [0, 0, 0, 0], [1 / 2, 0, 0, 1 / 2]])

1/2 * np.linalg.norm(np.sqrt(rho) @ np.sqrt(sigma), 1)

1.0000000000000002

## The `toqito` module

This is beyond the scope of this talk, but if you have an interest in quantum information, SDPs are a very useful tool to have in ones toolbox. The [toqito Python module]() makes use of SDPs to solve many problems that arise in this field. This is a non-exhaustive list of problems solved in`toqito` that is placed here for reference, and, to potentially spur some independent interest after this talk: 

- The SDP to calculate the quantum value of an XOR nonlocal game based on [arXiv:0608146](https://arxiv.org/abs/quant-ph/0608146).
- An approach to lower bound the quantum value of a nonlocal game can be achieved via the alternating projection algorithm proposed in [arXiv:0608128](https://arxiv.org/abs/quant-ph/0608128).
- The same lower bound technique mentioned above also holds for a more general class of nonlocal games based on [arXiv:1510.02083](https://arxiv.org/abs/1510.02083).
- The probability with which one can optimally clone a quantum state. This SDP is used, for instance, in [arXiv:1202.4010](https://arxiv.org/abs/1202.4010).
- The probability with which one can optimally distinguish a state from an ensemble using PPT measurements based on [arXiv:1205.1031](https://arxiv.org/abs/1205.1031).
- The probability with which one can optimally distinguish a state from an ensemble using global measurements based on [arXiv:0206093](https://arxiv.org/abs/quant-ph/0206093).
- The problem of state exclusion which is closely tied to the PBR theorem based on [arXiv:1111.3328]() and [arXiv:1306.4683](https://arxiv.org/abs/1306.4683).
- Determining whether a given quantum state possesses a symmetric extension can be phrased as an SDP [arXiv:0812.3607](https://arxiv.org/pdf/0812.3607.pdf).
- The output of the fidelity function may be phrased as the optimal value of an SDP based on [arXiv:0901.4709](https://arxiv.org/abs/0901.4709).
- The QIP(2) protocol can be phrased as an SDP, specifically with respect to [arXiv:1104.1140](https://arxiv.org/abs/1104.1140)

Furthermore, if you want to see more examples of how to numerically solve SDPs in Python, consulting the above list may be useful. Feel free to talk to me after this talk if you'd be interested in contributing to the project!

# References

[[Kiloran12]](https://uwspace.uwaterloo.ca/handle/10012/6662): Killoran, Nathan. "Entanglement quantification and quantum benchmarking of optical communication devices." (2012).

[[Watrous12]](https://arxiv.org/abs/1207.5726): Watrous, John. "Simpler semidefinite programs for completely bounded norms." arXiv preprint arXiv:1207.5726 (2012).

[[CVX]](http://cvxr.com/cvx/): Grant, Michael, Stephen Boyd, and Yinyu Ye. "CVX: Matlab software for disciplined convex programming." (2008).

[[cvxpy]](https://www.cvxpy.org/): Diamond, Steven, and Stephen Boyd. "CVXPY: A Python-embedded modeling language for convex optimization." The Journal of Machine Learning Research 17.1 (2016): 2909-2913.

[[toqito]](https://github.com/vprusso/toqito): Russo, Vincent "toqito: A {P}ython toolkit for quantum information." (2021).